In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from rdkit import Chem,DataStructs
from rdkit.Chem import(
    PandasTools,
    Draw,
    Descriptors,
    MACCSkeys,
    rdFingerprintGenerator,
)

## Generate canonical SMILES

In [ ]:
#DB에 있는 smi -> canonical
from rdkit import Chem
import os

def canonical_smiles_stream(file_name):
    line_num = 0
    canonical_list = []
    try:
        with open(file_name, 'r') as f:
            for line in f:
                line_num += 1
                
                # 줄 분리 및 SMILES 추출 (첫 번째 컬럼이라고 가정)
                parts = line.strip().split()
                if not parts: continue
                smi = parts[0]
                
                mol = Chem.MolFromSmiles(smi)
                
                if mol is not None:
                    # 유효하면 캐노니컬 SMILES로 변환하여 리스트에 추가
                    canonical_smi = Chem.MolToSmiles(mol) 
                    canonical_list.append(canonical_smi)
                else:
                    # 오류 발생 시 줄 번호 출력
                    print(f"❌ 파싱 오류! [줄 번호: {line_num}, SMILES: '{smi}']")

    except FileNotFoundError:
        print(f"❌ 오류: 파일 '{file_name}'을 찾을 수 없습니다.")
        return []

    print(f"\n--- 처리 완료 ---")
    print(f"총 처리 줄 수: {line_num}")
    print(f"✅ 성공 개수: {len(canonical_list)}개")


In [ ]:
#canonical_smi + ID
import pandas as pd
from rdkit import Chem

input = "EDB2M.smi" 
df = pd.read_csv(input, header=None, names=['SMILES',"ID"], sep='\s+', engine='python')


ids_data = df['ID'].tolist() 

output_file_name = "canonical_EDB2M.smi"
final_data_count = 0

with open(output_file_name, 'w') as f:
    for smi, compound_id in zip(Canon_SMILES, ids_data): 
        f.write(f"{smi}\t{compound_id}\n") 
        final_data_count += 1
        
print(f"✅ 저장 완료: 총 {final_data_count}개의 캐노니컬 SMILES와 ID가 '{output_file_name}'에 저장되었습니다.")

In [7]:
import pandas as pd

canonical = "canonical_EDB2M.smi"

df_loaded = pd.read_csv(canonical, sep='\t', header=None, names=['Canonical_SMILES', 'ID'])
df_loaded.head()

,Canonical_SMILES,ID
0,Cn1cc(I)cn1,Z1238965084
1,Brc1csc(Br)n1,Z1696844463
2,Brc1nnc(Br)s1,Z56871608
3,Ic1cnccn1,Z1695759744
4,Brc1csc(Br)c1,Z2050725166


In [8]:
#중복 제거
print(len(df_loaded))

dataset_new = df_loaded.drop_duplicates(subset=['Canonical_SMILES'])
len(dataset_new)

2013017


2013017

## ro5(Lipinski's Rule of Five) 기준 초기 선별

In [16]:
import math
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
import matplotlib.patches as mpatches
from rdkit import Chem
from rdkit.Chem import Descriptors, Draw, PandasTools

In [17]:
def calculate_ro5_properties(smi_id_pair):
    smi, compound_id = smi_id_pair
    molecule = Chem.MolFromSmiles(smi)
    if molecule is None:
        return None

    # Ro5 속성 계산 및 규칙 검사
    mw = Descriptors.ExactMolWt(molecule)
    n_hba = Descriptors.NumHAcceptors(molecule)
    n_hbd = Descriptors.NumHDonors(molecule)
    logp = Descriptors.MolLogP(molecule)
    ro5_fulfilled = sum([mw <= 500, n_hba <= 10, n_hbd <= 5, logp <= 5]) >= 3
    
    return (smi, compound_id, mw, n_hba, n_hbd, logp, ro5_fulfilled)

def process_ro5_from_file_simple(input_file_name, output_file_name):
    print(f"파일 '{input_file_name}' 로드 중...")
    
    # 1. 파일 로드 (Pandas)
    df = pd.read_csv(input_file_name, sep='\s+', header=None, names=['SMILES', 'ID'], usecols=[0, 1])
    
    # 2. 입력 데이터 준비: (SMILES, ID) 튜플 리스트
    input_data = list(zip(df['SMILES'].tolist(), df['ID'].tolist()))
    
    start_time = time.time()
    
    # 3. 순차 처리 (단일 코어 실행)
    results = [calculate_ro5_properties(item) for item in input_data]

    # 4. 결과 정리 및 Pandas DataFrame 생성
    valid_results = [r for r in results if r is not None] 
    
    results_df = pd.DataFrame(valid_results, 
                              columns=["canonical_smi", "ID", "MW", "n_HBA", "n_HBD", "LogP", "ro5_fulfilled"])
    
    # 5. 파일로 한 번에 저장 
    results_df.to_csv(
        output_file_name, 
        sep='\t',            
        index=False,         
        float_format='%.3f'  
    )
    
    # --- 요약 출력 ---
    print(f"총 {len(results_df)}개 데이터가 '{output_file_name}'에 저장되었습니다.")
    print(f"총 소요 시간: {time.time() - start_time:.2f}초")

INPUT_FILE = "canonical_EDB2M.smi" 
OUTPUT_FILE = "1st_screening.smi"

process_ro5_from_file_simple(INPUT_FILE, OUTPUT_FILE)


파일 'canonical_EDB2M.smi' 로드 중...
총 2013017개 데이터가 '1st_screening.smi'에 저장되었습니다.
총 소요 시간: 3515.02초


In [20]:
ro5 = pd.read_csv("1st_screening.smi", sep='\t')

ro5.head()

,canonical_smi,ID,MW,n_HBA,n_HBD,LogP,ro5_fulfilled
0,Cn1cc(I)cn1,Z1238965084,207.950,2,0,1.025,True
1,Brc1csc(Br)n1,Z1696844463,240.820,2,0,2.668,True
2,Brc1nnc(Br)s1,Z56871608,241.815,3,0,2.063,True
3,Ic1cnccn1,Z1695759744,205.934,2,0,1.081,True
4,Brc1csc(Br)c1,Z2050725166,239.824,1,0,3.273,True


In [ ]:
# Ro5 규칙을 통과한 화합물 (True인 경우)
molecules_ro5_fulfilled = ro5[ro5["ro5_fulfilled"]].copy()

# Ro5 규칙을 위반한 화합물 (False인 경우)
molecules_ro5_violated = ro5[~ro5["ro5_fulfilled"]].copy()

print(f"# compounds in unfiltered data set: {ro5.shape[0]}")
print(f"# compounds in filtered data set (Ro5 통과): {molecules_ro5_fulfilled.shape[0]}")
print(f"# compounds not compliant with the Ro5 (Ro5 위반): {molecules_ro5_violated.shape[0]}")

# NBVAL_CHECK_OUTPUT

# compounds in unfiltered data set: 2013017
# compounds in filtered data set (Ro5 통과): 2011428
# compounds not compliant with the Ro5 (Ro5 위반): 1589


In [ ]:
def calculate_veber_properties(smiles):
    molecule = Chem.MolFromSmiles(smiles)
    if molecule is None:
        # 오류 시 NaN 값 반환
        return pd.Series([None, None], index=["TPSA", "RBN"])

    # Veber 속성 계산
    tpsa = Descriptors.TPSA(molecule)
    rbn = Descriptors.NumRotatableBonds(molecule)
    
    # 결과를 Pandas Series로 반환
    return pd.Series([tpsa, rbn], index=["TPSA", "RBN"])


# --- 2. 메모리 내 DataFrame에 적용 ---

start_time = time.time()
print("Veber 속성 계산 시작...")

# 1. SMILES 컬럼에 함수 적용 (TPSA와 RBN 두 개의 컬럼이 반환됨)
veber_results = ro5["canonical_smi"].apply(calculate_veber_properties)

# 2. 계산된 결과를 기존 DataFrame에 병합
ro5 = pd.concat([ro5, veber_results], axis=1)

# 3. Veber 규칙 충족 여부 최종 판단 (두 기준 모두 만족: True)
ro5["veber_fulfilled"] = (ro5["TPSA"] <= 140) & (ro5["RBN"] <= 10)

end_time = time.time()

print(f"✅ Veber 속성 계산 및 추가 완료. (소요 시간: {end_time - start_time:.2f}초)")

# 필터링 결과를 확인합니다.
print("\n--- Ro5 + Veber 필터링 결과 ---")
df_final_passed = ro5[ro5['ro5_fulfilled'] & ro5['veber_fulfilled']].copy()
df_ro5_passed = ro5[ro5['ro5_fulfilled']].copy()

print(f"Ro5 통과 물질 수: {len(df_ro5_passed)}개")
print(f"Ro5 AND Veber 동시 통과 물질 수: {len(df_final_passed)}개")

Veber 속성 계산 시작...
✅ Veber 속성 계산 및 추가 완료. (소요 시간: 1709.36초)

--- Ro5 + Veber 필터링 결과 ---
Ro5 통과 물질 수: 2011428개
Ro5 AND Veber 동시 통과 물질 수: 2001252개


In [26]:
df_final_passed.to_csv("ro5_verber.smi", sep='\t', index = False, float_format='%.3f')

In [3]:
df_final_passed = pd.read_csv("ro5_verber.smi", sep='\t')

select = df_final_passed[['canonical_smi','ID']]